We should call `.py` so that cuda memory will be automatically released after each part.

This notebook is used to organize the codes

## Setup

### What you need to do

- Throw audio files into `data/` directory

- Correctly set `input_list`

- Create a `.env` file in the `process/` directory if using `glm-4`

In [2]:
import os

In [3]:
DATA_PATH = os.getcwd() + '/data/'
MODEL_PATH = '/ssdshare/LLMs/'
MUSIC_PATH = os.getcwd() + '/data/music/'
LLM_MODEL = "glm-4"
GENRATE_MODEL = "playground-v2.5-1024px-aesthetic"
CONTENT_PATH = DATA_PATH + '.tmp/generate/'
STYLE_PATH = DATA_PATH + 'style/illustration_style/'

if not os.path.exists(DATA_PATH + '.tmp/'):
  os.makedirs(DATA_PATH + '.tmp/')

list = ['extract/', 'generate/', 'process/', 'inprompt', 'style_transfer']

for folder in list:
  if not os.path.exists(DATA_PATH + '.tmp/' + folder):
    os.makedirs(DATA_PATH + '.tmp/' + folder)


In [4]:
input_list = [
  'NceS - Burn.mp3',
]
prompts = [r'''
  The name of this song is 'NceS - Burn'. 
''',
]
# Pick the style images in the style library
style_list = [
  # 'opia.png'
]
# You should check both input_list and prompts modified!!!
with open(DATA_PATH + 'input_list.txt', 'w') as f:
  for item in input_list:
    f.write("%s\n" % item)

with open(DATA_PATH + 'style_list.txt', 'w') as f:
  for item in style_list:
    f.write("%s\n" % item)

tmp_list = []
for item in input_list:
  tmp_list.append(item[:-4])
input_list = tmp_list

# if not os.path.exists(DATA_PATH + '.tmp/inprompt/'):
#   os.makedirs(DATA_PATH + '.tmp/inprompt/')
for (prompt, name) in zip(prompts, input_list):
  with open(DATA_PATH + '.tmp/inprompt/' + name + '.prompt', 'w') as f:
    f.write(prompt)

## Extract

In [16]:
os.system(f'python extract/extract.py --model_path {MODEL_PATH} --data_path {DATA_PATH} --music_path {MUSIC_PATH} --output_path {DATA_PATH}.tmp/extract/ --device_num 4')

NceS - Burn.mp3
['NceS - Burn.wav']
audio_start_id: 155163, audio_end_id: 155164, audio_pad_id: 151851.


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 9/9 [00:03<00:00,  2.36it/s]
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 9/9 [00:03<00:00,  2.38it/s]
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 9/9 [00:03<00:00,  2.38it/s]
The model is automatic

using device 0
using device 1
using device 2
using device 3
using device 0
using device 1
successfully add prompt for NceS - Burn.wav
successfully write prompt for NceS - Burn.wav


0

In [17]:
for file_name in input_list:
  with open(DATA_PATH + '.tmp/extract/' + file_name + '.prompt', 'r') as f:
    print(f.read())

This music is cut into 6 pieces. Each piece has a length of 30 seconds and an overlap of 5 seconds. The description of each piece is as follows:
Description piece 1: This piece of music is a pop song with a female vocalist. It has a light and airy feel, with a gentle acoustic guitar and soft drums providing the rhythmic background. The melody is catchy and easy to sing along to, with a repetitive chorus that creates a sense of unity and togetherness. The lyrics are about finding beauty and peace in the world around us, and the importance of living in the moment and embracing life's challenges. Overall, this song is a feel-good, uplifting anthem that encourages listeners to live their best lives and find happiness in the simple things.
Description piece 2: This is a song whose genre is Pop, and the lyrics are "So cold standing in the middle of the dark forest Oh my love because it's only you When you crash and burn".
Description piece 3: This is a high-energy electronic dance music piec

## Process

In [7]:
os.system(f'python process/process.py --model_path {MODEL_PATH} --data_path {DATA_PATH} --model {LLM_MODEL} --prompt_path {DATA_PATH}.tmp/extract/ --output_path {DATA_PATH}.tmp/process/')

['FULi AUTO SHOOTER']
Loading model
Model loaded
<class 'zhipuai._client.ZhipuAI'> <class 'NoneType'>
Token spent: 6699


0

In [7]:
for file_name in input_list:
  with open(DATA_PATH + '.tmp/process/' + file_name + '.prompt', 'r') as f:
    print(f.read())

dynamic cyberpunk cityscape, neon lights, futuristic vehicles, male protagonist in focus, intense expression, wearing high-tech gear, red and black color scheme, abstract patterns, strong shadows, glowing elements, sense of motion, urban chaos, 8k resolution, 16:9 aspect ratio, 60fps.


## Generate

In [8]:
os.system(f'python generate/generate.py --model_path {MODEL_PATH} --data_path {DATA_PATH} --model {GENRATE_MODEL} --output_path {DATA_PATH}.tmp/generate/ --prompt_path {DATA_PATH}.tmp/process/ --image_num 3')

Loading prompt from file
FULi AUTO SHOOTER.prompt
Prompt loaded
Loading model


Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  6.59it/s]


Model loaded
Generating for FULi AUTO SHOOTER.prompt


100%|██████████| 50/50 [00:16<00:00,  2.95it/s]


Generated for FULi AUTO SHOOTER.prompt
Generating image without characters
Loading prompt from file
Prompt loaded
Generating for FULi AUTO SHOOTER.prompt2


Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors
100%|██████████| 50/50 [00:16<00:00,  2.95it/s]


Generated for FULi AUTO SHOOTER.prompt2


0

# Style transfer

In [10]:
os.system(f'python style_transfer/style_transfer.py --data_path {DATA_PATH} --output_path {DATA_PATH}.tmp/style_transfer/ --style_path {STYLE_PATH} --content_path {CONTENT_PATH} -l_o')

Transferring from /root/LLM_project/codes/data/.tmp/generate/FULi AUTO SHOOTER/0.png to /root/LLM_project/codes/data/style/illustration_style/7.png
Building the style transfer model..


Style Loss : 5.652418 Content Loss: 1.724384:  54%|█████▎    | 161/300 [00:17<00:15,  8.93it/s] 